In [ ]:
%pip install pyautogen~=0.1.0 docker
%pip install agent-protocol



In [ ]:
import json

# Define the content you want to write to the JSON file
config_list = [
    {
        'model': 'gpt-4',
        'api_key': '<your OpenAI API key here>',
    },  # OpenAI API endpoint for gpt-4
    {
        'model': 'gpt-4',
        'api_key': '<your Azure OpenAI API key here>',
        'api_base': '<your Azure OpenAI API base here>',
        'api_type': 'azure',
        'api_version': '2023-07-01-preview',
    },  # Azure OpenAI API endpoint for gpt-4
    {
        'model': 'gpt-4-32k',
        'api_key': '<your Azure OpenAI API key here>',
        'api_base': '<your Azure OpenAI API base here>',
        'api_type': 'azure',
        'api_version': '2023-07-01-preview',
    },  # Azure OpenAI API endpoint for gpt-4-32k
]

# Specify the file name with a .json extension
file_name = "OAI_CONFIG_LIST.json"

# Write the content to the JSON file
with open(file_name, 'w') as file:
    json.dump(config_list, file, indent=4)

print(f"JSON file '{file_name}' has been created in the same folder as your notebook.")


In [ ]:
import autogen
from agent_protocol import Agent, Step, Task

#Use LLM
config_list = autogen.config_list_from_json(
    "OAI_CONFIG_LIST",
    filter_dict={
        "model": ["gpt-4", "gpt-4-0314", "gpt4", "gpt-4-32k", "gpt-4-32k-0314", "gpt-4-32k-v0314"],
    },
)

#Construct Agents:
planner = autogen.AssistantAgent(
    name="planner",
    llm_config={"config_list": config_list},
    # the default system message of the AssistantAgent is overwritten here
    system_message="You are a helpful AI assistant. You suggest coding and reasoning steps for another AI assistant to accomplish a task. Do not suggest concrete code. For any action beyond writing code or reasoning, convert it to a step which can be implemented by writing code. For example, the action of browsing the web can be implemented by writing code which reads and prints the content of a web page. Finally, inspect the execution result. If the plan is not good, suggest a better plan. If the execution is wrong, analyze the error and suggest a fix."
)
planner_user = autogen.UserProxyAgent(
    name="planner_user",
    max_consecutive_auto_reply=0,  # terminate without auto-reply
    human_input_mode="NEVER",
)

def ask_planner(message):
    planner_user.initiate_chat(planner, message=message)
    # return the last message received from the planner
    return planner_user.last_message()["content"]

# create an AssistantAgent instance named "assistant"
assistant = autogen.AssistantAgent(
    name="assistant",
    llm_config={
        "temperature": 0,
        "request_timeout": 600,
        "seed": 42,
        "model": "gpt-4-0613",
        "config_list": autogen.config_list_openai_aoai(exclude="aoai"),
        "functions": [
            {
                "name": "ask_planner",
                "description": "ask planner to: 1. get a plan for finishing a task, 2. verify the execution result of the plan and potentially suggest new plan.",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "message": {
                            "type": "string",
                            "description": "question to ask planner. Make sure the question include enough context, such as the code and the execution result. The planner does not know the conversation between you and the user, unless you share the conversation with the planner.",
                        },
                    },
                    "required": ["message"],
                },
            },
        ],
    }
)

# create a UserProxyAgent instance named "user_proxy"
user_proxy = autogen.UserProxyAgent(
    name="user_proxy",
    human_input_mode="TERMINATE",
    max_consecutive_auto_reply=10,
    # is_termination_msg=lambda x: "content" in x and x["content"] is not None and x["content"].rstrip().endswith("TERMINATE"),
    code_execution_config={"work_dir": "planning"},
    function_map={"ask_planner": ask_planner},
)

import autogen
from agent_protocol import Agent, Step, Task

# ... (Your code for creating agents and functions)

# Agent - Protocol Wrapper
async def plan(step: Step) -> Step:
    task = await Agent.db.get_task(step.task_id)

    # You need to implement generete_steps function to generate steps from the task input
    steps = generate_steps(task.input)

    last_step = steps[-1]
    for step_name in steps[:-1]:
        await Agent.db.create_step(task_id=task.task_id, name=step_name, ...)

    await Agent.db.create_step(task_id=task.task_id, name=last_step, is_last=True)
    step.output = steps
    return step

async def execute(step: Step) -> Step:
    # Implement code execution logic here
    # Use tools, websearch, etc.
    # Update step.output with the execution result
    return step

async def task_handler(task: Task) -> None:
    await Agent.db.create_step(task_id=task.task_id, name="plan", ...)

async def step_handler(step: Step) -> Step:
    if step.name == "plan":
        return await plan(step)
    else:
        return await execute(step)

Agent.setup_agent(task_handler, step_handler).start()

# Now, initiate a chat with the user_proxy agent to start the conversation
user_proxy.initiate_chat(
    assistant,
    message="""Suggest a fix to an open good first issue of flaml""",
)